In [1]:
import random
import socket
import struct
import numpy as np
import copy
import hashlib
import intervals
from pprint import pprint

In [2]:
numberOfBitsOfHash = 3

In [3]:
# fourthNode.findPredecessor(fourthNode.nodeId-2).nodeId
# contains()

In [30]:
class Node:
    def __init__(self, nodeId):
        self.nodeId = nodeId
        self.fingerTable = None
        self.predecessor=None
        self.successor=[nodeId]
    
    def findClosestPreceedingFinger(self,id):
        for i in range(numberOfBitsOfHash-1,-1,-1):
            if(contains(self.fingerTable[i].node.nodeId,self.nodeId,id,numberOfBitsOfHash,2)):
                return self.fingerTable[i].node
        return self   
    
    def findPredecessor(self,id):
        idCopy=self
        while (not contains(id,idCopy.nodeId,idCopy.successor.nodeId,numberOfBitsOfHash,3)):
            idCopy=idCopy.findClosestPreceedingFinger(id)
        return idCopy
    
    def findSuccessor(self,id):
        predecessor = self.findPredecessor(id)
        return predecessor.successor
    
    #Joining a node
    def updateFingerTable(self,node,i):
        if contains(node.nodeId,self.nodeId,self.fingerTable[i].node.nodeId,numberOfBitsOfHash,2):
            self.fingerTable[i].node = node
            p = self.predecessor
            p.updateFingerTable(node,i)

    def updateOthers(self):
        for i in range(1,numberOfBitsOfHash+1):
            pred = self.findPredecessor((self.nodeId - pow(2,i-1))%pow(2,numberOfBitsOfHash))
            pred.updateFingerTable(self,i-1)

    def initFingerTable(self,existingNode):
        self.fingerTable[0].node = existingNode.findSuccessor(self.fingerTable[0].start)
        self.successor = self.fingerTable[0].node
        self.predecessor = self.successor.predecessor
        self.successor.predecessor = self
        self.predecessor.successor = self
        for i in range(0,numberOfBitsOfHash-1):
            if contains(self.fingerTable[i+1].start,self.nodeId,self.fingerTable[i].node.nodeId,numberOfBitsOfHash,4):
                self.fingerTable[i+1].node = self.fingerTable[i].node
            else:
                self.fingerTable[i+1].node = existingNode.findSuccessor(self.fingerTable[i+1].start)

    def join(self,existingNode):
        self.initFingerTable(existingNode)
        self.updateOthers()
        
    #Lookup Query
    def lookup(self,key):
        requiredNode = self.findSuccessor(key)
        return requiredNode.nodeId
    
    #Node Failure
    def updateFingerTableOnFailure(self,node,i):
        print("Hyioyy")
        if(node.nodeId==self.fingerTable[i].node.nodeId):
            print("Hiii")
            self.fingerTable[i].node=node.successor
        elif contains(node.nodeId,self.nodeId,self.fingerTable[i].node.nodeId,numberOfBitsOfHash,2):
            print("yo")
            self.fingerTable[i].node = node.successor
            p = self.predecessor
            p.updateFingerTableOnFailure(node,i)

    def updateOthersOnFailure(self):
        for i in range(1,numberOfBitsOfHash+1):
#             print(":h")
            pred = self.findPredecessor((self.nodeId - pow(2,i-1))%pow(2,numberOfBitsOfHash))
            print(pred.nodeId)
            pred.updateFingerTableOnFailure(self,i-1)
            
    def nodeFailure(self):
        self.updateOthersOnFailure()
        self.predecessor.successor=self.successor
        self.successor.predecessor=self.predecessor

    
    #Stabilization of Network
    
    def notify(self,origNode):
        if self.predecessor or contains(origNode.nodeId,self.predecessor.nodeId,self.nodeId,numberOfBitsOfHash,2):
            self.predecessor=origNode
            
    def stabilize(self):
        x = self.successor.predecessor
        if contains(x.nodeId,self.nodeId,self.successor.nodeId,numberOfBitsOfHash,2):
            self.successor=x
        self.successor.notify(self)

    
    def fixFingerTables(self):
        for i in range(0,numberOfBitsOfHash):
            self.fingerTable[i].node = self.findSuccessor(self.fingerTable[i].start)
            

In [32]:
firstNode = Node(0)
firstNode = createChordRing(firstNode)
secondNode=initializeNewNode(2)
thirdNode=initializeNewNode(5)
fourthNode=initializeNewNode(7)
nodeDict={0:firstNode,2:secondNode,5:thirdNode}
for index,i in enumerate(firstNode.fingerTable):
    if contains(i.start,firstNode.nodeId,secondNode.nodeId,numberOfBitsOfHash,1):
        firstNode.fingerTable[index].node=secondNode

for index,i in enumerate(secondNode.fingerTable):
    if contains(i.start,secondNode.nodeId,firstNode.nodeId,numberOfBitsOfHash,1):
        secondNode.fingerTable[index].node=firstNode
print("----------Initializing 2 Nodes(0,2)-------------")        
firstNode.successor=secondNode
firstNode.predecessor=secondNode
secondNode.successor=firstNode
secondNode.predecessor=firstNode
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("----------Adding 1 node(5)-------------")        
thirdNode.join(firstNode)
# thirdNode.fixFingerTables()
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("--------------")
printNodeAttributes(thirdNode)
print("----------Adding 1 node(7)-------------")        
fourthNode.join(firstNode)
# thirdNode.fixFingerTables()
# fourthNode.fixFingerTables()
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("--------------")
printNodeAttributes(thirdNode)
print("--------------")
printNodeAttributes(fourthNode)
# print("----------Removing 1 node(5)-------------")
thirdNode.nodeFailure()
# # del thirdNode
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("--------------")
printNodeAttributes(fourthNode)
print("--------------")

----------Initializing 2 Nodes(0,2)-------------
Node Id: 0
Node Successor: 2
Node Predecessor: 2
Start: 1  Successor: 2
Start: 2  Successor: 2
Start: 4  Successor: 0
--------------
Node Id: 2
Node Successor: 0
Node Predecessor: 0
Start: 3  Successor: 0
Start: 4  Successor: 0
Start: 6  Successor: 0
----------Adding 1 node(5)-------------
Node Id: 0
Node Successor: 2
Node Predecessor: 5
Start: 1  Successor: 2
Start: 2  Successor: 2
Start: 4  Successor: 5
--------------
Node Id: 2
Node Successor: 5
Node Predecessor: 0
Start: 3  Successor: 5
Start: 4  Successor: 5
Start: 6  Successor: 0
--------------
Node Id: 5
Node Successor: 0
Node Predecessor: 2
Start: 6  Successor: 0
Start: 7  Successor: 0
Start: 1  Successor: 2
----------Adding 1 node(7)-------------
Node Id: 0
Node Successor: 2
Node Predecessor: 7
Start: 1  Successor: 2
Start: 2  Successor: 2
Start: 4  Successor: 5
--------------
Node Id: 2
Node Successor: 5
Node Predecessor: 0
Start: 3  Successor: 5
Start: 4  Successor: 5
Start: 6

In [14]:
def contains(key1,lowerBound,upperBound,numberOfBitsOfHash,flag):
    if lowerBound==upperBound:
        return 1
    if(flag==1): #Closed Interval
        if key1==lowerBound or key1==upperBound:
            return 1
    elif(flag==2): #Open Interval
        if key1==lowerBound or key1==upperBound:
            return 0
    elif(flag==3): #open closed interval
        if key1==lowerBound:
            return 0
        elif key1==upperBound:
            return 1
    elif(flag==4): #closed open interval
        if key1==lowerBound:
            return 1
        elif key1==upperBound:
            return 0
    

    if lowerBound<upperBound:
        diff = upperBound - lowerBound
        temp1 = key1 - lowerBound
        temp2 = upperBound-key1
        if temp1<diff and temp2<diff:
            return 1
        else:
            return 0
    elif lowerBound>upperBound:
        temp=lowerBound
        lowerBound=upperBound
        upperBound=temp
        diff = upperBound - lowerBound
        temp1 = key1 - lowerBound
        temp2 = upperBound-key1
        if temp1<diff and temp2<diff:
            return 0
        else:
            return 1
        
# def distance( n1, n2,numberOfBitsOfHash):
#     print("n1",n1)
#     print("n2",n2)
#     if n1 == n2:
#         return 0
#     if n1 < n2:
#         return n2 - n1
#     return pow(2,numberOfBitsOfHash) - n1 + n2

# def contains(a,lb,ub,numberOfBitsOfHash,temp):
#     if distance(lb,a,numberOfBitsOfHash) <= distance(ub, a,numberOfBitsOfHash):
#         return 1
#     return 0

In [6]:
def initializeFingerTableOfFirstNode(firstNode):
    start_list=[]
    interval_list=[]
    successorList=[]
    for k in range(1,numberOfBitsOfHash+1):
        start=(firstNode.nodeId+pow(2,k-1))%(pow(2,numberOfBitsOfHash))
        end=(firstNode.nodeId+pow(2,k)-1)%(pow(2,numberOfBitsOfHash))
        interval={"start":start%(pow(2,numberOfBitsOfHash)),"end":(end)%(pow(2,numberOfBitsOfHash))}
        start_list.append(start)
        interval_list.append(interval)
        successorList.append(firstNode)
    
    fingerTable = np.rec.fromarrays((start_list,interval_list,successorList ), names=('start', 'interval','node'))  

    return fingerTable

def createChordRing(firstNode):
    firstNode.successor=firstNode
    firstNode.fingerTable=initializeFingerTableOfFirstNode(firstNode)
    firstNode.predecessor = firstNode
    return firstNode


def initializeNewNode(newNodeId):
    newNode = Node(newNodeId)
    newNode.successor = firstNode.findSuccessor(newNode.nodeId)
    newNode.fingerTable = initializeFingerTableOfFirstNode(newNode)
    return newNode

def printNodeAttributes(nodeValue):
    print("Node Id:",nodeValue.nodeId)
    print("Node Successor:",nodeValue.successor.nodeId)
    print("Node Predecessor:",nodeValue.predecessor.nodeId)
    for i in nodeValue.fingerTable:
        print("Start:",i.start," Successor:",i.node.nodeId)
        

In [ ]:
firstNode = Node(0)
firstNode = createChordRing(firstNode)
secondNode=initializeNewNode(2)
# thirdNode=initializeNewNode(5)


In [ ]:
for index,i in enumerate(firstNode.fingerTable):
    if contains(i.start,firstNode.nodeId,secondNode.nodeId,numberOfBitsOfHash,1):
        firstNode.fingerTable[index].node=secondNode

for index,i in enumerate(secondNode.fingerTable):
    if contains(i.start,secondNode.nodeId,firstNode.nodeId,numberOfBitsOfHash,1):
        secondNode.fingerTable[index].node=firstNode
        
firstNode.successor=secondNode
firstNode.predecessor=secondNode
secondNode.successor=firstNode
secondNode.predecessor=firstNode
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("--------------")

In [ ]:
thirdNode.join(firstNode)
# fourthNode.join(firstNode)

In [ ]:
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("--------------")
printNodeAttributes(thirdNode)
print("--------------")


In [ ]:
thirdNode.nodeFailure()

In [ ]:
printNodeAttributes(firstNode)
print("--------------")
printNodeAttributes(secondNode)
print("--------------")
printNodeAttributes(thirdNode)
print("--------------")
# printNodeAttributes(fourthNode)

In [ ]:
thirdNode.findPredecessor(4).nodeId

In [ ]:
thirdNode.successor.nodeId